In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
import numpy as np
import pandas as pd
import os

from torch.utils.data import DataLoader, Dataset
import torch.utils.data as utils
from torchvision import transforms

import torch
import torch.nn as nn
import torch.optim as optim 
import torchvision


import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline


In [ ]:
!unzip /kaggle/input/aerial-cactus-identification/train.zip


In [ ]:
!unzip /kaggle/input/aerial-cactus-identification/test.zip


In [ ]:
data_directory = '/kaggle/working/'
train_directory = data_directory + 'train/'
test_directory = data_directory + 'test/'


In [ ]:
labels = pd.read_csv("/kaggle/input/aerial-cactus-identification/train.csv")
labels.head()


In [ ]:
class ImageData(Dataset):
    def __init__(self, df, data_directory, transform):
        super().__init__()
        self.df = df
        self.data_directory = data_directory
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):       
        img_name = self.df.id[index]
        label = self.df.has_cactus[index]
        
        img_path = os.path.join(self.data_directory, img_name)
        image = mpimg.imread(img_path)
        image = self.transform(image)
        return image, label


In [ ]:
data_transf = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor()])
train_data = ImageData(df = labels, data_directory = train_directory, transform = data_transf)
train_loader = DataLoader(dataset = train_data, batch_size = 64)


In [ ]:
!pip install efficientnet_pytorch


In [ ]:
from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_name('efficientnet-b1')


In [ ]:
# Unfreezing model weights
for param in model.parameters():
    param.requires_grad = True


In [ ]:
num_ftrs = model._fc.in_features
model._fc = nn.Linear(num_ftrs, 1)


In [ ]:
model = model.to('cuda')


In [ ]:
optimizer = optim.NAdam(model.parameters())


In [ ]:
loss_func = nn.BCELoss()


In [ ]:
%%time
# Training model for 5 epochs and logging the time required for the entire training process
loss_log = []

for epoch in range(5):    
    model.train()    
    for ii, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        target = target.float()                
        target = target.unsqueeze(1)
        
        optimizer.zero_grad()
        output = model(data)                
    
        m = nn.Sigmoid()
        loss = loss_func(m(output), target)
        loss.backward()

        optimizer.step()  
        
        if ii % 1000 == 0:
            loss_log.append(loss.item())
       
    print('Epoch Number : {} - Loss Value: {:.6f}'.format(epoch + 1, loss.item()))
    
    
    
print('Training time :')


In [ ]:
plt.figure(figsize=(10,8))
plt.title("Model Log Loss")
plt.xlim(0,4)
plt.ylim(0,1)
plt.xscale("linear")
plt.xlabel("Epochs")
plt.ylabel("Log Loss")
plt.plot(loss_log)


In [ ]:
submission = pd.read_csv('/kaggle/input/aerial-cactus-identification/sample_submission.csv')


In [ ]:
test_data = ImageData(df = submission, data_directory = test_directory, transform = data_transf)
test_loader = DataLoader(dataset = test_data, shuffle=False)


In [ ]:
%%time
predict = []
model.eval()
for i, (data, _) in enumerate(test_loader):
    data = data.cuda()
    output = model(data)    

    pred = torch.sigmoid(output)
    predicted_vals = pred > 0.5
    predict.append(int(predicted_vals))
    
submission['has_cactus'] = predict
submission.to_csv('submission.csv', index=False)

print('Testing time :')


In [ ]:
submission.head()
